In [1]:
# EX1: The n-dimensional tensor mastery challenge: Combine the `Head` and `MultiHeadAttention` 
#      into one class that processes all the heads in parallel, treating the heads as another 
#      batch dimension (answer is in nanoGPT). (
#      nanoGPT: https://github.com/karpathy/nanoGPT/blob/master/model.py

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [148]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5001
eval_interval = 500
learning_rate = 3e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 2
dropout = 0.2

In [19]:
torch.manual_seed(1337)

In [20]:
with open('../input.txt', 'r', encoding='utf-8') as f: 
    text = f.read()

In [21]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from character to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [22]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [23]:
# data loading
def get_batch(split):
    # generate a small batch of data inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [80]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [58]:
# Original

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.H = head_size
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        Hs = self.H
        B, T, C = x.shape
        k = self.key(x) # (B, T, Hs)
        q = self.query(x) # (B, T, Hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * Hs**-0.5 # (B, T, Hs) @ (B, Hs, T) ---> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, Hs)
        out = wei @ v # (B, T, T) @ (B, T, Hs) ---> (B, T, Hs)
        return out
    
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [177]:
# Merged MultiHeadAttention

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.Hn = num_heads
        self.Hs = head_size
        self.attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # key, query and value as one batch
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        Hs = self.Hs
        Hn = self.Hn
        B, T, C = x.shape
        q, k, v = self.attn(x.float()).split(n_embd, dim=-1)
        q = q.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        k = k.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        v = v.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * Hs**-0.5 # (B, Hn, T, Hs) @ (B, Hn, Hs, T) ---> (B, Hn, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, Hn, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        out = wei @ v # (B, Hn, T, T) @ (B, Hn, T, Hs) ---> (B, Hn, T, Hs)

        out = out.transpose(1, 2).contiguous().view(B, T, C) # (B, Hn, T, Hs) ---> (B, T, Hn * Hs)
        out = self.proj(out) # from MHA
        out = self.dropout(out) # from MHA
        return out

In [178]:
class FeedForward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd lile
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class GPTModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd=n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [179]:
model = GPTModel()
m = model.to(device)

In [180]:
def train():
    print(f"Parameter count: {sum(p.numel() for p in model.parameters())/1e6} M parameters")
    
    optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

    for iter in range(max_iters):

        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter >= 42:
            losses = estimate_loss(model)
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = m(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        if iter >= 42:
            break

In [181]:
# Train with modified code
train()

Parameter count: 0.112193 M parameters
step 0: train loss 4.3302, val loss 4.3270
step 42: train loss 3.3718, val loss 3.3980


In [88]:
# Train with original code
train()

Parameter count: 0.112193 M parameters
step 0: train loss 3.0655, val loss 3.1043
step 42: train loss 2.8720, val loss 2.9045


In [ ]:
# Conclusion: As we can see from this example the new MultiHeadAttention improves the speed of the training by ~13%